# Support Vector Regression - SVM

In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [67]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [68]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [69]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [70]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [71]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [72]:
## dependent asd independent features
X = df.drop(labels = "total_bill", axis = 1)
y = df['total_bill']

In [73]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=10)

In [74]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2


In [75]:
## Encoding the binary categories by label encoding
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [76]:
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

In [77]:
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

In [78]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [79]:
## Applying OneHotEncoding technique for multi categories
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [80]:
ct = ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop = 'first'),['day'])],remainder='passthrough')

In [81]:
X_train = ct.fit_transform(X_train)

In [82]:
X_test = ct.transform(X_test)

##### Starting to apply support vector regressor

In [83]:
from sklearn.svm import SVR
svr = SVR()

In [86]:
svr.fit(X_train,y_train)
pred = svr.predict(X_test)
pred

array([16.63987262, 17.80422691, 14.49145093, 24.62866424, 14.67173361,
       22.67410166, 19.04664805, 28.29108519, 23.75972181, 21.46546953,
       24.86896212, 17.67206073, 13.44339231, 22.32034304, 19.75694037,
       14.95399101, 13.8566174 , 13.39304164, 21.2171004 , 17.63874958,
       14.32207633, 15.49171819, 14.19580813, 13.3369827 , 29.65235934,
       13.8566174 , 17.37720431, 21.38260597, 14.76962827, 30.02183062,
       28.64203024, 28.03548476, 14.79770992, 15.92806049, 17.56421584,
       13.35938349, 20.52423608, 14.51606288, 20.71506125, 22.18900247,
       16.70814899, 21.52705566, 13.2124233 , 24.44552357, 14.05257741,
       14.13643952, 18.52010744, 14.79770992, 13.74271464, 18.54833482,
       20.82086506, 29.78470039, 19.41806679, 27.8646883 , 13.01786739,
       16.91813134, 14.06890087, 14.15277698, 17.87180292, 13.83656792,
       17.37317545])

In [89]:
from sklearn.metrics import mean_absolute_error, r2_score
print("Mean Absolute error: ", mean_absolute_error(y_test, pred))
print("r2 score: ", r2_score(y_test,pred))

Mean Absolute error:  4.1486423210190235
r2 score:  0.46028114561159283


##### Aplying hyperparameter tuning using Gridsearchcv

In [91]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C':[0.,1,10,100,1000],
    'gamma' : [1,0.1,0.01,0.001,0.0007],
    'kernel' : ['rbf']
}
grid = GridSearchCV(SVR(),param_grid=param_grid, refit=True, cv = 5, verbose=True)

In [92]:
grid.fit(X_train,y_train)
pred = grid.predict(X_test)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
25 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
  File "c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", 

In [93]:
print("Mean Absolute error: ", mean_absolute_error(y_test, pred))
print("r2 score: ", r2_score(y_test,pred))

Mean Absolute error:  3.879290668577761
r2 score:  0.5056655724069592
